# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
weather_df = pd.read_csv('../wheather_dataframe.csv')
weather_df.head()

,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed
0,2122389.0,Ossora,RU,59.24,163.07,07/18/20,56.26,78.0,100.0,10.47
1,2017155.0,Saskylakh,RU,71.92,114.08,07/18/20,53.01,58.0,44.0,10.31
2,4030556.0,Rikitea,PF,-23.12,-134.97,07/18/20,70.23,82.0,98.0,14.56
3,2077963.0,Albany,AU,-35.02,117.89,07/18/20,55.81,72.0,100.0,5.39
4,1509955.0,Bobrovka,RU,54.05,61.75,07/18/20,70.72,84.0,36.0,6.51


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:

# Store latitude and longitude in locations
lat_lon = weather_df[['latitude', 'longitude']]

# Fill NaN values and convert to float
humid = weather_df["humidity"]
lat_lon['latitude'].value_counts()
nanindex = lat_lon.loc[lat_lon['latitude'].isnull()].index
#lat_lon['latitude'].isnull().values.sum()
weather_df.drop(nanindex,inplace=True)
weather_df

,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed
0,2122389.0,Ossora,RU,59.24,163.07,07/18/20,56.26,78.0,100.0,10.47
1,2017155.0,Saskylakh,RU,71.92,114.08,07/18/20,53.01,58.0,44.0,10.31
2,4030556.0,Rikitea,PF,-23.12,-134.97,07/18/20,70.23,82.0,98.0,14.56
3,2077963.0,Albany,AU,-35.02,117.89,07/18/20,55.81,72.0,100.0,5.39
4,1509955.0,Bobrovka,RU,54.05,61.75,07/18/20,70.72,84.0,36.0,6.51
...,...,...,...,...,...,...,...,...,...,...
619,2027196.0,Balakhninskiy,RU,58.01,114.28,07/18/20,59.07,63.0,100.0,4.18
620,2176639.0,Batemans Bay,AU,-35.72,150.18,07/18/20,66.99,46.0,54.0,1.01
621,1060673.0,Manjakandriana,MG,-18.92,47.80,07/18/20,51.80,76.0,90.0,14.99
622,4013679.0,Coahuayana Viejo,MX,18.73,-103.68,07/18/20,78.66,83.0,100.0,1.39


In [23]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create heat layer
heat_layer = gmaps.heatmap_layer(lat_lon, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
# A max temperature lower than 80 degrees but higher than 70.
crit_temperature = (weather_df.temp < 80) & (weather_df.temp > 70)
crit_windSpeed = weather_df.wind_speed < 15
crit_cloudiness = weather_df.cloudiness == 0
final_criteria = crit_temperature & crit_windSpeed & crit_cloudiness 

# Use boolean indexing to filter the weather_df dataframe
ideal_weather_df = weather_df[final_criteria]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head(10)

,index,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed
0,7,2126199.0,Cherskiy,RU,68.75,161.30,07/18/20,71.22,48.0,0.0,8.16
1,55,127349.0,Khorramabad,IR,33.49,48.36,07/18/20,79.86,33.0,0.0,3.60
2,64,3578441.0,Saint-François,GP,16.25,-61.27,07/18/20,77.00,83.0,0.0,3.36
3,162,1805733.0,Jinchang,CN,38.50,102.17,07/18/20,78.66,17.0,0.0,3.11
4,193,2209055.0,Waddān,LY,29.16,16.14,07/18/20,75.58,54.0,0.0,6.20
5,230,2065594.0,Mount Isa,AU,-20.73,139.50,07/18/20,78.80,11.0,0.0,6.93
6,252,1798760.0,Pingliang,CN,35.54,106.69,07/18/20,75.40,37.0,0.0,4.56
7,298,3465228.0,Coxim,BR,-18.51,-54.76,07/18/20,70.72,41.0,0.0,5.10
8,355,2527087.0,Tiznit Province,MA,29.58,-9.50,07/18/20,78.08,30.0,0.0,1.54
9,372,2146219.0,Hervey Bay,AU,-25.30,152.85,07/18/20,73.00,55.0,0.0,5.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
hotel_df = ideal_weather_df
# Add column for Hotel Name - Note that we used "" to specify initial entry.
hotel_df['Hotel Name'] = ""
hotel_df.head()

,index,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed,Hotel Name
0,7,2126199.0,Cherskiy,RU,68.75,161.30,07/18/20,71.22,48.0,0.0,8.16,
1,55,127349.0,Khorramabad,IR,33.49,48.36,07/18/20,79.86,33.0,0.0,3.60,
2,64,3578441.0,Saint-François,GP,16.25,-61.27,07/18/20,77.00,83.0,0.0,3.36,
3,162,1805733.0,Jinchang,CN,38.50,102.17,07/18/20,78.66,17.0,0.0,3.11,
4,193,2209055.0,Waddān,LY,29.16,16.14,07/18/20,75.58,54.0,0.0,6.20,


In [27]:
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df.info()

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         19 non-null     int64  
 1   city_id       19 non-null     float64
 2   city_name     19 non-null     object 
 3   country       19 non-null     object 
 4   latitude      19 non-null     float64
 5   longitude     19 non-null     float64
 6   weather_date  19 non-null     object 
 7   temp          19 non-null     float64
 8   humidity      19 non-null     float64
 9   cloudiness    19 non-null     float64
 10  wind_speed    19 non-null     float64
 11  Hotel Name    19 non-null     object 
dtypes: float64(7), int64(1), object(4)
memory usage: 1.9+ KB


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>city_name</dt><dd>{city_name}</dd>
<dt>country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [34]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…